In [1]:
import numpy as np
import dill
from lkf_tools.dataset import *

In [2]:
lkf_data = dill.load(open("data/lkf_pickle_files/lkf_data_2015_005.pickle", "rb"))

In [3]:
lkfs_all = []
for it in lkf_data.indexes:
    lkfs_path = lkf_data.lkfpath.joinpath('lkf_%s_%03i.npy'
                                          % (lkf_data.netcdf_file.split('/')[-1].split('.')[0],
                                             (it+1))
                                         )
    lkfs_all.append(np.load(lkfs_path, allow_pickle=True))

In [4]:
print('day   number of lkfs')
for ind, item in enumerate(lkfs_all):
    print(ind+1, '   ', np.shape(item)[0])

day   number of lkfs
1     90
2     63
3     54
4     68
5     90
6     79
7     71
8     73
9     77


In [5]:
tracks_all = []

for it in lkf_data.indexes:
    tracks_path = lkf_data.track_output_path.joinpath('lkf_tracked_pairs_%s_to_%s.npy'
                                                      % (lkf_data.lkf_filelist[it][4:-4],
                                                         lkf_data.lkf_filelist[it+1][4:-4])
                                                     )
    tracks_all.append(np.load(tracks_path, allow_pickle=True))

In [6]:
print('\n'.join(['\t'.join([str(x[i]) if len(x)>i else '' for x in tracks_all]) for i in range(20)]))

[4 2]	[1 3]	[2 3]	[1 2]	[0 0]	[1 2]	[0 0]	[0 0]	[0 1]
[8 8]	[2 5]	[ 3 62]	[ 3 11]	[7 4]	[ 4 56]	[1 1]	[2 3]	[ 3 67]
[9 4]	[ 5 11]	[4 6]	[4 4]	[11  6]	[11  6]	[2 2]	[ 4 72]	[4 3]
[12 10]	[ 6 47]	[8 9]	[10 12]	[14  9]	[12 65]	[4 5]	[5 4]	[5 4]
[13  7]	[ 6 50]	[12 12]	[11  9]	[15 14]	[14  8]	[ 6 17]	[9 6]	[8 5]
[14  9]	[ 8 15]	[14 15]	[12 18]	[18 11]	[16 57]	[ 6 69]	[15 11]	[13 69]
[17  8]	[12 17]	[19 20]	[15 74]	[27 27]	[19 10]	[ 7 14]	[18 20]	[14  9]
[17 59]	[14 18]	[20 66]	[15 84]	[27 72]	[22 16]	[ 8 63]	[19 14]	[18 10]
[20 13]	[14 19]	[21 22]	[16 15]	[34 37]	[23 60]	[10 23]	[21 18]	[20 70]
[21 18]	[15 22]	[22 19]	[18 21]	[39 45]	[24 14]	[11 64]	[22 22]	[21 12]
[23 16]	[19 20]	[23 25]	[20 24]	[39 60]	[26 24]	[14 26]	[35 27]	[27 20]
[25 17]	[23 25]	[24 25]	[21 22]	[40 44]	[27 23]	[15 28]	[36 28]	[30 20]
[28 14]	[23 29]	[25 26]	[26 26]	[41 74]	[30 26]	[18 31]	[37 30]	[30 28]
[30 19]	[24 26]	[26 27]	[27 31]	[42 39]	[35 42]	[22 29]	[37 33]	[34 30]
[31 21]	[26 27]	[30 28]	[28 38]	[46 53]	[3

In [7]:
##### here only one of the paths is saved if the lkf has
# two associated features in the following record #####

# lkf_tracks[i] are the lkf tracks that start at day i+1
lkf_tracks = np.zeros(len(lkfs_all),dtype='object')

# number all the lkfs on day one
for startday in range(len(lkfs_all)):
    lkf_tracks[startday] = np.arange(np.shape(lkfs_all[startday])[0], dtype='object')

    # get the lkf number on day two (if the lkf can be tracked)
    # and add it to the lkf
    for ind, item in enumerate(lkf_tracks[startday]):
        in_next_record = (item == tracks_all[startday][:,0])

        if np.any(in_next_record):
            pos_in_next_record = np.where(in_next_record)
            lkf_number_in_next_record = tracks_all[startday][pos_in_next_record,1]

            lkf_tracks[startday][ind] = np.append(item, lkf_number_in_next_record[0,0])

    # loop over the following days
    for i in range(1,len(tracks_all)-startday):
        for ind, item in enumerate(lkf_tracks[startday]):
            if np.array(item).size == i+1:
                in_next_record = (item[-1] == tracks_all[i+startday][:,0])

                if np.any(in_next_record):
                    pos_in_next_record = np.where(in_next_record)

                    lkf_number_in_next_record = tracks_all[i+startday][pos_in_next_record,1]

                    lkf_tracks[startday][ind] = np.append(item, lkf_number_in_next_record[0,0])

In [8]:
already_tracked = np.zeros(len(lkf_tracks)-1,dtype='object')
for it in range(len(lkf_tracks)-1):
    
    for ind, item in enumerate(lkf_tracks[it]):

        if np.array(item).size > 1:
            already_tracked[it] = [np.array(item).flat[1] if already_tracked[it]==0 else np.append(already_tracked[it],np.array(item).flat[1])]
            
for i in range(1,len(tracks_all)):
    lkf_tracks[i] = np.delete(lkf_tracks[i], already_tracked[i-1])

In [9]:
lifetimes = np.zeros_like(lkf_tracks, dtype='object')
for i in range(len(lkf_tracks)):
    lifetimes[i] = np.zeros_like(lkf_tracks[i], dtype='object')
    for ind, item in enumerate(lkf_tracks[i]):
        lifetimes[i][ind] = np.array(item).size

In [11]:
total_mean_lifetime = np.concatenate(lifetimes).mean()
total_mean_lifetime

1.7582938388625593

In [12]:
lifetimes_tracked_lkfs = [lifetimes[i][np.where(lifetimes[i]!=1)] for i in range(len(lifetimes))]
total_mean_lifetime_tracked_lkfs = np.concatenate(lifetimes_tracked_lkfs).mean()
total_mean_lifetime_tracked_lkfs

2.8181818181818183